In [40]:
import cvxpy as cp
import numpy as np
# Define and solve the CVXPY problem.
n = 10
A = np.random.randn(2, n)
u = np.random.randn(2, 1)
y = cp.Variable(n)
Ay = A @ y
obj = cp.sum_squares(Ay) + u.T@Ay
constraints =  [cp.norm2(y[:5]) <= 1, cp.norm2(y[5:]) <= 1]
prob = cp.Problem(cp.Minimize(obj), constraints)
prob.solve(solver=cp.GUROBI, verbose=False)
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(y.value)


The optimal value is -0.2144246575688528
The optimal x is
[-0.01952547  0.00183893  0.03520406  0.04025137  0.00313448  0.06676809
 -0.08108803  0.06316108  0.02849691 -0.0072229 ]


# test dual

In [47]:
import argparse
import sys
sys.path.append("../..")
from src.funcs.regularizer import  NatOG
from src.utils import gen_chain_group
import numpy as np

def get_config():
    parser = argparse.ArgumentParser()
    # IPG solver configurations
    parser.add_argument("--ipg_save_log", default=True, type=lambda x: (str(x).lower()
                        in ['true', '1', 'yes']), help="whether save the log of the ipg solver.")
    parser.add_argument("--exact_pg_computation", default=False, type=lambda x: (str(x).lower()
                        in ['true', '1', 'yes']), help="Mimic to the exact computation of the proximal operator.") 
    parser.add_argument("--exact_pg_computation_tol", type=float, default=1e-15, help="Deisred error tolerance.")                                                                      

    parser.add_argument("--ipg_do_linesearch", default=True, type=lambda x: (str(x).lower()
                        in ['true', '1', 'yes']), help="Whether do linesearch in the projected gradient ascent.")
    parser.add_argument("--ipg_linesearch_eta", type=float, default=1e-4, help="eta of the linesearch.")
    parser.add_argument("--ipg_linesearch_xi", type=float, default=0.8, help="xi of the linesearch.")
    parser.add_argument("--ipg_linesearch_beta", type=float, default=1.2, help="beta of the linesearch.")
    parser.add_argument("--ipg_linesearch_limits", type=int, default=100, help="max attempts of the linesearch.")
    
    parser.add_argument("--ipg_strategy", type=str, default="diminishing", choices=["diminishing"], 
        help="Strategy to inexactly evaluate the proximal operator.\ndiminishing: c * np.log(k+1) / k**delta")
    parser.add_argument("--ipg_diminishing_c", type=float, default=1, help="c of c * np.log(k+1) / k**delta")
    parser.add_argument("--ipg_diminishing_delta", type=float, default=2, help="delta of c * np.log(k+1) / k**delta")
    config = parser.parse_args("")
    return config

In [100]:
groups = [[0,1,2], [1,2,3], [0,2,4]]
print('using:', groups)
config = get_config()
r = NatOG(groups, penalty=1.0, config=config, weights=np.array([1,100,1]))
print(r.A.todense())
print(r.xdim)
np.random.seed(0)
x0 = np.random.randn(r.xdim, 1)
print(r.func(x0))
dk=np.random.randn(r.xdim, 1)
alphak = 1.0
xtrial, ytrial = r.compute_inexact_proximal_gradient_update(xk=x0, alphak=alphak, dk=dk, 
                                            y_init=None, stepsize_init=None, 
                                            ipg_kwargs={'iteration':100000})
print(f'target duality gap:{r.targap:.3f} | subsolver status:{r.flag} | aoptim:{r.aoptim:.3f}')
print('init x             :', x0.T)
print('approxmate solution:', xtrial.T)
Aytrial = r.A @ ytrial
dual = alphak/2 * Aytrial.T@Aytrial + uk.T@Aytrial
print('dual               :', dual)

using: [[0, 1, 2], [1, 2, 3], [0, 2, 4]]
[[1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
5
252.58908176102412
target duality gap:0.000 | subsolver status:desired | aoptim:2.805
init x             : [[1.76405235 0.40015721 0.97873798 2.2408932  1.86755799]]
approxmate solution: [[0.97437783 0.         0.         0.         0.81525549]]
dual               : [[-7.54898474]]


In [101]:
y = cp.Variable(ytrial.shape)
uk = x0 - alphak * dk
A = r.A
Ay = A @ y
obj = alphak/2 * cp.sum_squares(Ay) + uk.T@Ay
constraints =  [cp.norm2(y[start:end]) <= radius for start, end, radius in zip(r.starts, r.ends, r.weights)]
prob = cp.Problem(cp.Minimize(obj), constraints)
prob.solve(solver=cp.GUROBI, verbose=False, max_iters=1000)
print("\nThe optimal value is", prob.value)
print(y.value.T)


The optimal value is -7.548983723326806
[[-9.99999886e-01 -4.76447820e-05  6.03685322e-05  5.50224975e-01
  -1.13065780e+00 -2.34527903e+00 -7.67013696e-01  1.51030187e-05
  -6.41630650e-01]]


In [102]:
prob.value - dual.item(), np.linalg.norm(ytrial - y.value)

(1.013372272318236e-06, 0.001328561619588002)